# DICOM to NifTI

DICOM is the international standard to store medical imaging information. It is flexible in that it allows to store all the data necessary for different acquisition protocols (modality, devices and workflows), which produce different metadata and image pixel data types.

A medical imaging dataset can be subdivided in series of DICOM files, each series corresponding to a single patient's data. We can first write a function that loads one of these series into memory. In python, we can use the [pydicom](https://pydicom.github.io/pydicom/stable/) library to work with DICOM data:

In [1]:
#|default_exp dicom_to_nifti

In [2]:
#|export
import os
import pydicom

def dicom_serie_load(series_base_path, serie_uid):

    serie_path = f"{series_base_path}/{serie_uid}"
    instances_filename = os.listdir(serie_path)
    n_instances = len(instances_filename)

    ds_l = [None] * n_instances
    for i, instance_filename in enumerate(instances_filename):
        ds_l[i] = pydicom.dcmread(f"{serie_path}/{instance_filename}")

    return ds_l

In [3]:
base_path_dicom = f"{os.environ["RSNA_IAD_DATA_DIR"]}"
series_path_dicom = f"{base_path_dicom}/series"

series_uid_l = os.listdir(series_path_dicom)
ds_l = dicom_serie_load(series_path_dicom, series_uid_l[0])
len(ds_l), ds_l[0].SOPInstanceUID

(188, '1.2.826.0.1.3680043.8.498.10124807242473374136099471315028464450')

Now we have list of `pydicom.Dataset` objects, corresponding to a (3D) volume. This volume can be contained in a single DICOM file (so the returned list of the function above will have only one element) or in multiple DICOM files, each one containing a single slice. A slice is a thin or cross-sectional image, created by scanning the body at a specific position along a chosen axis. Stacking these slices, according to that position, results in a volume. One simple way of doing this is obtaining the position of each of the slices along the z axis, sorting them by that position, and stacking them. We can obtain that position from the "Image Position (Patient)" attribute of the DICOM data:

In [4]:
#|export
def dicom_get_zposition(ds):
    
    if getattr(ds, "ImagePositionPatient", None) and len(ds.ImagePositionPatient) >= 3:
        z_position = ds.ImagePositionPatient[2]
    else:  # in case the tag is missing or does not contain the z axis position
        z_position = ds.InstanceNumber

    return float(z_position)

In [5]:
dicom_get_zposition(ds_l[0]), dicom_get_zposition(ds_l[1])

(38.935824681372, 32.460949707916)

>Note: from now on i'm going to refer to data individual values as voxels (values in 3D space) instead of pixels (values in 2D space).

Also, the stored voxel values may not be in real-world units (like Hounsfield Units in CT imaging). So to convert those we need to use the "Rescale Slope" and "Rescale Intercept" attributes in the DICOM data to linearly transform the stored voxel values:

In [16]:
#|export
def dicom_get_rescale_factors(ds):

    slope = getattr(ds, "RescaleSlope", 1.0)
    intercept = getattr(ds, "RescaleIntercept", 0.0)
    
    return float(slope), float(intercept)

In [17]:
dicom_get_rescale_factors(ds_l[0]), dicom_get_rescale_factors(ds_l[1])

((1.0, 0.0), (1.0, 0.0))

The drawback of DICOM files is that they are slow to load from disk, because of all the flexibility to handle the different types of data, which makes parsing this data more computationally expensive. So they are not suitable for working with relatively large amounts of medical imaging data on a regular basis, like for deep learning experimentation. For that, a less flexible standard called NifTI is used. Each NifTI file has to contain a whole volume and may contain only a minimal set of metadata attributes to interpret and spatially place the volume's data, i.e.:
    - the volume's shape.
    - the space between voxels in real-world distance units.
    - an affine transformation matrix, which maps the volumes voxel data in voxel coordinates to real-world coordinates.
    - the scaling slope/intercept to linearly scale the voxel values.
    - ...

We could use the [dicom2nifti](https://github.com/icometrix/dicom2nifti) package to transform a whole series of DICOM files to a NifTI file automatically. However, since during inference we do want to load the DICOM files and go straight for the data transforms in predictions, and not have an intermediate step of saving them as NifTI files, we will write the functions to get the metadata we need from the DICOM, without the need of saving it as a NifTI file.

First, we get the pixel/voxel array from the DICOM data (which is usually compressed) into an `torch.Tensor`, and we remove that data from a copy of the DICOM data (so we do not modify the original one). So we separate the array from the metadata:

In [18]:
#|export
import torch
from copy import deepcopy

def dicom_split_array_from_metadata(ds):  # process a single DICOM

    ds_copy = deepcopy(ds)  # copy the dicom object
    pixel_pt = torch.from_numpy(ds_copy.pixel_array)
    del ds_copy.PixelData  # remove pixel data from the dicom object copy
    
    return pixel_pt, ds_copy

In [19]:
from objsize import get_deep_size
slice_pt, ds = dicom_split_array_from_metadata(ds_l[0])
print(slice_pt.shape, get_deep_size(ds))

torch.Size([512, 512]) 65880


Now we can write a function that takes the list of `pydicom.Dataset` objects containing the DICOM data from a series and outputs a volume `torch.Tensor` and the `pydicom.Dataset` object without the pixel data, meaning, containing only the metadata:

In [20]:
#|export
def dicom_serie_process(ds_l):

    n_ds = len(ds_l)

    pixel_pt, ds_metadata = dicom_split_array_from_metadata(ds_l[0])  # Get first instance

    if n_ds == 1:  # one dicom with the whole volume
        volume = pixel_pt
        ds_metadata_l = [ds_metadata]
    else:  # each dicom with a slice
        volume = torch.zeros((n_ds, *pixel_pt.shape), dtype=torch.float32)
        ds_metadata_l = [None] * n_ds
        volume[0] = pixel_pt
        ds_metadata_l[0] = ds_metadata

        # To later sort the slices
        zpositions = torch.zeros((n_ds,), dtype=torch.float32)
        zpositions[0] = dicom_get_zposition(ds_l[0])
        
        for i, ds in enumerate(ds_l[1:], start=1):
            volume[i], ds_metadata_l[i] = dicom_split_array_from_metadata(ds)
            zpositions[i] = dicom_get_zposition(ds)

		# sort slices in the volume
        zpositions_argsort = torch.argsort(zpositions)
        volume = volume[zpositions_argsort]
	
    # rescale volume
    slope, intercept = dicom_get_rescale_factors(ds_l[0])
    volume = volume * slope + intercept
    
    return volume, ds_metadata_l

In [21]:
volume, ds_metadata_l = dicom_serie_process(ds_l)
print(volume.shape, get_deep_size(ds_metadata_l))

torch.Size([188, 512, 512]) 12156321


We want the spacing between voxels to be similar, since the deep learning model that processes the volume will not capture that information. To normalize that spacing, we first need to obtain it from the DICOM data:

In [22]:
#|export
def dicom_get_spacing(ds):

    pixel_spacing = getattr(ds, "PixelSpacing", None)
    slice_thickness = getattr(ds, "SliceThickness", None)

    if (pixel_spacing is None) or (slice_thickness is None):
        shared_functional_groups_sequence = getattr(ds, "SharedFunctionalGroupsSequence", None)
        if shared_functional_groups_sequence is not None:
            pixel_measures_sequence = getattr(shared_functional_groups_sequence[0], "PixelMeasuresSequence", None)
            if pixel_measures_sequence is not None:
                if pixel_spacing is None:
                    pixel_spacing = getattr(pixel_measures_sequence[0], "PixelSpacing", None)
                if slice_thickness is None:
                    slice_thickness = getattr(pixel_measures_sequence[0], "SliceThickness", None)
    
    if pixel_spacing is None or slice_thickness is None:
        raise Exception("Missing either Pixel Spacing or Slice Thickness.")
    
    pixel_spacing = [float(axis_spacing) for axis_spacing in pixel_spacing]
    slice_thickness = float(slice_thickness)
    spacing = [*pixel_spacing, slice_thickness]

    return spacing

In [23]:
dicom_get_spacing(ds_metadata_l[0])

[0.3515625, 0.3515625, 0.5]

In [24]:
#|export
def dicom_serie_get_spacing(ds_l):

    spacings = torch.zeros((len(ds_l), 3), requires_grad=False)
    for i, ds in enumerate(ds_l):
        spacings[i] = torch.tensor(dicom_get_spacing(ds))

    return spacings.mode(dim=0).values

In [25]:
dicom_serie_get_spacing(ds_metadata_l)

tensor([0.3516, 0.3516, 0.5000])

Finally, to store the data locally, we can use `nibabel` to save the volume (here we do not computed the affine transformation, since we won't use it), and the voxel spacing as a compressed NifTI file:

In [26]:
#|export
import numpy as np
import nibabel

def dicom_volume_to_nifti(volume, ds_metadata_l, serie_uid, path):

    nifti_path = f"{path}/{serie_uid}.nii.gz"

    nifti = nibabel.nifti1.Nifti1Image(volume.detach().cpu().numpy(), affine=np.eye(4))
    spacing = dicom_serie_get_spacing(ds_metadata_l)    
    nifti.header.set_zooms(spacing)
    
    nibabel.save(nifti, nifti_path)

In [27]:
base_path_nifti = f"{os.environ["RSNA_IAD_DATA_DIR"]}/nifti"
series_path_nifti = f"{base_path_nifti}/series"
if not os.path.exists(series_path_nifti):
    os.mkdir(series_path_nifti)

dicom_volume_to_nifti(volume, ds_metadata_l, series_uid_l[0], series_path_nifti)

In [28]:
#|export
def dicom_serie_to_nifti(base_path_dicom, serie_uid, base_path_nifti):
    ds_l = dicom_serie_load(base_path_dicom, serie_uid)
    volume, ds_metadata_l = dicom_serie_process(ds_l)
    dicom_volume_to_nifti(volume, ds_metadata_l,  serie_uid, base_path_nifti)

In [29]:
dicom_serie_to_nifti(series_path_dicom, series_uid_l[0], series_path_nifti)

To load and convert multiple DICOM series in parallel, we can use the `ProcessPoolExecutor`:

In [30]:
#|export
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing
from tqdm import tqdm

def dicom_series_to_niftis(base_path_dicoms, series_uid, base_path_niftis, max_workers):

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(dicom_serie_to_nifti, base_path_dicoms, serie_uid, base_path_niftis) for serie_uid in series_uid]
        for future in tqdm(as_completed(futures), total=len(futures)):
            ...

In [31]:
import random

random.seed(0)
samples_series_uid_l = random.sample(series_uid_l, 5)
dicom_series_to_niftis(series_path_dicom, samples_series_uid_l, series_path_nifti, max_workers=5) # max_workers=multiprocessing.cpu_count() to use all cores

100%|█████████████████████████████████████████████████████████████| 5/5 [00:49<00:00,  9.99s/it]


So in this notebook, we cover the basics of how to load and process DICOM files data, and store that for analysis and experimentation, particularly, using the NifTI format.

In [32]:
from nbdev.export import nb_export

In [33]:
nb_export("1_dicom_to_nifti.ipynb", "../lib")